# Building an NHL xG Model

In [1]:
import pandas as pd
import hockey_scraper
import numpy as np
import math
import seaborn as sns
from tqdm.notebook import tqdm # This displays a loading bar for monitoring progress of for loops

<br>

## Scraping data from 2022 Regular Season

In [2]:
# This code scrapes the play-by-play data of the 2022-23 NHL season. 
# It takes a while so I saved the data as a csv file on my local computer

# scraped_data = hockey_scraper.scrape_seasons([2022], False, data_format='Pandas')

In [3]:
data = pd.read_csv('pbp2022.csv')

<br>

## Data Prep

In [4]:
# Selecting columns to fit the model on and columns to use for model evaluation

model_columns = ['Game_Id','Date','Period','Event','Description','Seconds_Elapsed','Strength','Ev_Zone','Type','xC','yC','Away_Team','Home_Team']
eval_columns = ['Ev_Team','p1_name', 'p1_ID', 'p2_name','p2_ID', 'p3_name', 'p3_ID', 'awayPlayer1', 'awayPlayer1_id',
                'awayPlayer2', 'awayPlayer2_id', 'awayPlayer3', 'awayPlayer3_id','awayPlayer4', 'awayPlayer4_id',
                'awayPlayer5', 'awayPlayer5_id','awayPlayer6', 'awayPlayer6_id', 'homePlayer1', 'homePlayer1_id',
                'homePlayer2', 'homePlayer2_id', 'homePlayer3', 'homePlayer3_id','homePlayer4', 'homePlayer4_id', 
                'homePlayer5', 'homePlayer5_id','homePlayer6', 'homePlayer6_id','Away_Goalie', 'Away_Goalie_Id', 
                'Home_Goalie', 'Home_Goalie_Id']

In [5]:
# Only shots from Periods 1, 2 and 3 and OTs
shots = data.loc[(data.Period == 1) | (data.Period == 2) | (data.Period == 3) | (data.Period == 4)]
# Selecting shot events
shots = shots.loc[(shots.Event == 'SHOT') | (shots.Event == 'MISS') | (shots.Event == 'BLOCK') | (shots.Event == 'GOAL')]
# Removing penalty shots
shots = shots.drop(index = shots.loc[shots.Strength == '0x0'].index)
# Selecting only regular season games
shots = shots.loc[shots.Game_Id < 30000].sort_values(by=['Game_Id','Period'],ascending=[True,True]).reset_index(drop=True)
shots

,Game_Id,Date,Period,Event,Description,Time_Elapsed,Seconds_Elapsed,Strength,Ev_Zone,Type,...,Away_Score,Home_Score,Away_Goalie,Away_Goalie_Id,Home_Goalie,Home_Goalie_Id,xC,yC,Home_Coach,Away_Coach
0,20001,2022-10-07,1,SHOT,"SJS ONGOAL - #28 MEIER, Wrist, Off. Zone, 45 ft.",0:23,23,5x5,Off,WRIST SHOT,...,0,0,JAMES REIMER,8473503.0,JUUSE SAROS,8477424.0,44.0,8.0,John Hynes,David Quinn
1,20001,2022-10-07,1,MISS,"SJS #44 VLASIC, Wrist, Wide of Net, Off. Zone,...",0:36,36,5x5,Off,WRIST SHOT,...,0,0,JAMES REIMER,8473503.0,JUUSE SAROS,8477424.0,44.0,27.0,John Hynes,David Quinn
2,20001,2022-10-07,1,BLOCK,"NSH #27 MCDONAGH BLOCKED BY SJS #62 LABANC, W...",0:56,56,5x5,Def,WRIST SHOT,...,0,0,JAMES REIMER,8473503.0,JUUSE SAROS,8477424.0,-55.0,3.0,John Hynes,David Quinn
3,20001,2022-10-07,1,SHOT,"NSH ONGOAL - #14 EKHOLM, Slap, Off. Zone, 56 ft.",0:59,59,5x5,Off,SLAP SHOT,...,0,0,JAMES REIMER,8473503.0,JUUSE SAROS,8477424.0,-33.0,8.0,John Hynes,David Quinn
4,20001,2022-10-07,1,GOAL,"NSH #44 SHERWOOD(1), Wrist, Off. Zone, 15 ft.A...",1:01,61,5x5,Off,WRIST SHOT,...,0,0,JAMES REIMER,8473503.0,JUUSE SAROS,8477424.0,-74.0,-5.0,John Hynes,David Quinn
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
152630,21312,2023-04-13,3,SHOT,"SEA ONGOAL - #67 GEEKIE, Wrist, Off. Zone, 10 ft.",17:14,1034,5x5,Off,WRIST SHOT,...,2,1,LAURENT BROSSOIT,8476316.0,PHILIPP GRUBAUER,8475831.0,81.0,7.0,Dave Hakstol,Bruce Cassidy
152631,21312,2023-04-13,3,MISS,"VGK #9 EICHEL, Wrist, Wide of Net, Def. Zone, ...",18:33,1113,5x5,Def,WRIST SHOT,...,2,1,LAURENT BROSSOIT,8476316.0,NaN,NaN,55.0,41.0,Dave Hakstol,Bruce Cassidy
152632,21312,2023-04-13,3,BLOCK,"SEA #4 SCHULTZ BLOCKED BY VGK #3 MCNABB, Wris...",18:43,1123,5x5,Def,WRIST SHOT,...,2,1,LAURENT BROSSOIT,8476316.0,NaN,NaN,75.0,-1.0,Dave Hakstol,Bruce Cassidy
152633,21312,2023-04-13,3,GOAL,"VGK #20 STEPHENSON(16), Poke, Def. Zone, 137 ft.",19:22,1162,5x5,Def,NaN,...,2,1,LAURENT BROSSOIT,8476316.0,NaN,NaN,47.0,19.0,Dave Hakstol,Bruce Cassidy


In [6]:
# Removing shots with NaN locations from ML dataset

array = []

for i in tqdm(range(len(shots))):
    if math.isnan(shots.at[i,'xC']) == True:
        array.append(i)
        
shots = shots.drop(index = array).reset_index(drop = True)
shots

  0%|          | 0/152635 [00:00<?, ?it/s]

,Game_Id,Date,Period,Event,Description,Time_Elapsed,Seconds_Elapsed,Strength,Ev_Zone,Type,...,Away_Score,Home_Score,Away_Goalie,Away_Goalie_Id,Home_Goalie,Home_Goalie_Id,xC,yC,Home_Coach,Away_Coach
0,20001,2022-10-07,1,SHOT,"SJS ONGOAL - #28 MEIER, Wrist, Off. Zone, 45 ft.",0:23,23,5x5,Off,WRIST SHOT,...,0,0,JAMES REIMER,8473503.0,JUUSE SAROS,8477424.0,44.0,8.0,John Hynes,David Quinn
1,20001,2022-10-07,1,MISS,"SJS #44 VLASIC, Wrist, Wide of Net, Off. Zone,...",0:36,36,5x5,Off,WRIST SHOT,...,0,0,JAMES REIMER,8473503.0,JUUSE SAROS,8477424.0,44.0,27.0,John Hynes,David Quinn
2,20001,2022-10-07,1,BLOCK,"NSH #27 MCDONAGH BLOCKED BY SJS #62 LABANC, W...",0:56,56,5x5,Def,WRIST SHOT,...,0,0,JAMES REIMER,8473503.0,JUUSE SAROS,8477424.0,-55.0,3.0,John Hynes,David Quinn
3,20001,2022-10-07,1,SHOT,"NSH ONGOAL - #14 EKHOLM, Slap, Off. Zone, 56 ft.",0:59,59,5x5,Off,SLAP SHOT,...,0,0,JAMES REIMER,8473503.0,JUUSE SAROS,8477424.0,-33.0,8.0,John Hynes,David Quinn
4,20001,2022-10-07,1,GOAL,"NSH #44 SHERWOOD(1), Wrist, Off. Zone, 15 ft.A...",1:01,61,5x5,Off,WRIST SHOT,...,0,0,JAMES REIMER,8473503.0,JUUSE SAROS,8477424.0,-74.0,-5.0,John Hynes,David Quinn
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
152504,21312,2023-04-13,3,SHOT,"SEA ONGOAL - #67 GEEKIE, Wrist, Off. Zone, 10 ft.",17:14,1034,5x5,Off,WRIST SHOT,...,2,1,LAURENT BROSSOIT,8476316.0,PHILIPP GRUBAUER,8475831.0,81.0,7.0,Dave Hakstol,Bruce Cassidy
152505,21312,2023-04-13,3,MISS,"VGK #9 EICHEL, Wrist, Wide of Net, Def. Zone, ...",18:33,1113,5x5,Def,WRIST SHOT,...,2,1,LAURENT BROSSOIT,8476316.0,NaN,NaN,55.0,41.0,Dave Hakstol,Bruce Cassidy
152506,21312,2023-04-13,3,BLOCK,"SEA #4 SCHULTZ BLOCKED BY VGK #3 MCNABB, Wris...",18:43,1123,5x5,Def,WRIST SHOT,...,2,1,LAURENT BROSSOIT,8476316.0,NaN,NaN,75.0,-1.0,Dave Hakstol,Bruce Cassidy
152507,21312,2023-04-13,3,GOAL,"VGK #20 STEPHENSON(16), Poke, Def. Zone, 137 ft.",19:22,1162,5x5,Def,NaN,...,2,1,LAURENT BROSSOIT,8476316.0,NaN,NaN,47.0,19.0,Dave Hakstol,Bruce Cassidy


<br>

## Engineering Features for Distance and Change of Angle

The function below returns the short change and long change net coordinates for both home and away teams. This was necessary because the home and away teams start at different ends in different arenas. 

In [7]:
def get_net_locations(df,game_id):

    temp = df.loc[(df.Game_Id == game_id) & (df.Event == 'SHOT')].reset_index(drop=True)
    for i in range(len(temp)):
        if (temp.at[i,'Ev_Zone'] == 'Neu') | (temp.at[i,'Ev_Zone'] == 'Def'):
            continue
        if temp.at[i,'Ev_Zone'] == 'Off':
            if temp.at[i,'Ev_Team'] == temp.at[i,'Away_Team']:
                if temp.at[i,'xC'] > 0:
                    home_short_change_net_coord = (89,0)
                    away_short_change_net_coord = (-89,0)
                    home_long_change_net_coord = (-89,0)
                    away_long_change_net_coord = (89,0)
                    break
                if temp.at[i,'xC'] < 0:
                    home_short_change_net_coord = (-89,0)
                    away_short_change_net_coord = (89,0)
                    home_long_change_net_coord = (89,0)
                    away_long_change_net_coord = (-89,0)
                    break
            if temp.at[i,'Ev_Team'] == temp.at[i,'Home_Team']:
                if temp.at[i,'xC'] > 0:
                    home_short_change_net_coord = (-89,0)
                    away_short_change_net_coord = (89,0)
                    home_long_change_net_coord = (89,0)
                    away_long_change_net_coord = (-89,0)
                    break
                if temp.at[i,'xC'] < 0:
                    home_short_change_net_coord = (89,0)
                    away_short_change_net_coord = (-89,0)
                    home_long_change_net_coord = (-89,0)
                    away_long_change_net_coord = (89,0)
                    break
                    
    return home_short_change_net_coord,away_short_change_net_coord,home_long_change_net_coord,away_long_change_net_coord

In [8]:
# Testing cell. 

hscn_coord, ascn_coord, hlcn_coord, alcn_coord = get_net_locations(shots,20777)
print('The Home Short Change Net Location is:', hscn_coord)
print('The Away Short Change Net Location is:', ascn_coord)
print('The Home Long Change Net Location is:', hlcn_coord)
print('The Away Long Change Net Location is:', alcn_coord)

The Home Short Change Net Location is: (89, 0)
The Away Short Change Net Location is: (-89, 0)
The Home Long Change Net Location is: (-89, 0)
The Away Long Change Net Location is: (89, 0)


In [9]:
# Sanity check for above

shots.loc[shots.Game_Id == 20777].head(n=5)

,Game_Id,Date,Period,Event,Description,Time_Elapsed,Seconds_Elapsed,Strength,Ev_Zone,Type,...,Away_Score,Home_Score,Away_Goalie,Away_Goalie_Id,Home_Goalie,Home_Goalie_Id,xC,yC,Home_Coach,Away_Coach
90197,20777,2023-01-26,1,SHOT,"CHI ONGOAL - #11 RADDYSH, Slap, Off. Zone, 46 ft.",0:15,15,5x5,Off,SLAP SHOT,...,0,0,JAXSON STAUBER,8483530.0,JACOB MARKSTROM,8474593.0,48.0,-21.0,Darryl Sutter,Luke Richardson
90198,20777,2023-01-26,1,SHOT,"CHI ONGOAL - #43 BLACKWELL, Backhand, Off. Zon...",1:52,112,5x5,Off,BACKHAND,...,0,0,JAXSON STAUBER,8483530.0,JACOB MARKSTROM,8474593.0,82.0,3.0,Darryl Sutter,Luke Richardson
90199,20777,2023-01-26,1,SHOT,"CHI ONGOAL - #52 JOHNSON, Snap, Off. Zone, 36 ft.",2:34,154,5x5,Off,SNAP SHOT,...,0,0,JAXSON STAUBER,8483530.0,JACOB MARKSTROM,8474593.0,74.0,-33.0,Darryl Sutter,Luke Richardson
90200,20777,2023-01-26,1,SHOT,"CHI ONGOAL - #51 MITCHELL, Wrist, Off. Zone, 3...",3:59,239,5x5,Off,WRIST SHOT,...,0,0,JAXSON STAUBER,8483530.0,JACOB MARKSTROM,8474593.0,57.0,16.0,Darryl Sutter,Luke Richardson
90201,20777,2023-01-26,1,SHOT,"CGY ONGOAL - #73 TOFFOLI, Wrist, Off. Zone, 41...",4:09,249,5x5,Off,WRIST SHOT,...,0,0,JAXSON STAUBER,8483530.0,JACOB MARKSTROM,8474593.0,-53.0,-20.0,Darryl Sutter,Luke Richardson


From the example above, the results from the `get_net_locations` function matches the information from the dataframe. 

Now to create a `Distance` variable using the information from the function above. The function below takes the necessary information from each shot event type and returns the distance from the shot location to the centre of the goal. 

In [10]:
def get_shot_distances(df,game_id, period, team, away_team, home_team, x_coord, y_coord):
    
    hscn_coord, ascn_coord, hlcn_coord, alcn_coord = get_net_locations(df,game_id)
    
    if (period == 1) | (period == 3):
        if team == away_team:
            distance = math.sqrt((hscn_coord[0]-x_coord)**2 + (hscn_coord[1]-y_coord)**2) # yC for the net is always zero
        if team == home_team:
            distance = math.sqrt((ascn_coord[0]-x_coord)**2 + (ascn_coord[1]-y_coord)**2) # yC for the net is always zero
            
    if (period == 2) | (period == 4):
        if team == away_team:
            distance = math.sqrt((hlcn_coord[0]-x_coord)**2 + (hlcn_coord[1]-y_coord)**2) # yC for the net is always zero
        if team == home_team:
            distance = math.sqrt((alcn_coord[0]-x_coord)**2 + (alcn_coord[1]-y_coord)**2) # yC for the net is always zero
            
    return distance

Done! Now to create a `Shot Angle` feature. This feature will describe from what angle the shot is taken from. Deciding not to implement a `Change of Angle` feature since the NHL play-by-play data does not include pass event data therefore the change of angle from the previous recorded play will not be representative of goaltender pre-shot movement.

In [11]:
def get_shot_angles(df,game_id, period, team, away_team, home_team, x_coord, y_coord):
    
    hscn_coord, ascn_coord, hlcn_coord, alcn_coord = get_net_locations(df,game_id)
    
    if (period == 1) | (period == 3):
        if hscn_coord[0] > 0:
            if team == away_team:
                if x_coord == 89: # To deal with divide by zero issues
                    angle = 90
                else:
                    angle = math.degrees(math.atan(abs(y_coord)/(hscn_coord[0]-x_coord))) 
            if team == home_team:
                if x_coord == -89: # To deal with divide by zero issues
                    angle = 90
                else:
                    angle = math.degrees(math.atan(abs(y_coord)/(x_coord-ascn_coord[0])))
        else:
            if team == away_team:
                if x_coord == -89: # To deal with divide by zero issues
                    angle = 90
                else:
                    angle = math.degrees(math.atan(abs(y_coord)/(x_coord-hscn_coord[0]))) 
            if team == home_team:
                if x_coord == 89: # To deal with divide by zero issues
                    angle = 90
                else:    
                    angle = math.degrees(math.atan(abs(y_coord)/(ascn_coord[0]-x_coord)))
            
    if (period == 2) | (period == 4):
        if hlcn_coord[0] > 0:
            if team == away_team:
                if x_coord == 89: # To deal with divide by zero issues
                    angle = 90
                else:
                    angle = math.degrees(math.atan(abs(y_coord)/(hlcn_coord[0]-x_coord))) 
            if team == home_team:
                if x_coord == -89: # To deal with divide by zero issues
                    angle = 90
                else:
                    angle = math.degrees(math.atan(abs(y_coord)/(x_coord-alcn_coord[0])))
        else:
            if team == away_team:
                if x_coord == -89: # To deal with divide by zero issues
                    angle = 90
                else:
                    angle = math.degrees(math.atan(abs(y_coord)/(x_coord-hlcn_coord[0]))) 
            if team == home_team:
                if x_coord == 89: # To deal with divide by zero issues
                    angle = 90
                else:
                    angle = math.degrees(math.atan(abs(y_coord)/(alcn_coord[0]-x_coord)))
            
    return angle

The calculations for `Distance` and `Shot Angle` are in the following cell. The calculations take a while so they are saved in another csv.

In [12]:
# for j in tqdm(range(len(shots))):
#     shots.at[j,'Distance'] = get_shot_distances(shots,shots.at[j,'Game_Id'],shots.at[j,'Period'],shots.at[j,'Ev_Team'],
#                                                  shots.at[j,'Away_Team'],shots.at[j,'Home_Team'],shots.at[j,'xC'],
#                                                  shots.at[j,'yC'])
#     shots.at[j,'Shot_Angle'] = get_shot_angles(shots,shots.at[j,'Game_Id'],shots.at[j,'Period'],shots.at[j,'Ev_Team'],
#                                                  shots.at[j,'Away_Team'],shots.at[j,'Home_Team'],shots.at[j,'xC'],
#                                                  shots.at[j,'yC'])
    
# shots

<br>

## Score Differential Input

Most public models have score difference as an input. Logically, this should not make sense but I will engineer this feature so that I can input it to the model. 

In [13]:
def score_diff(team, away_team, home_team, away_score, home_score):
    
    if team == home_team:
        if abs(home_score-away_score) >= 3:
            if home_score > away_score:
                diff = '3+'
            else:
                diff = '-3-'
        else:
            diff = str(home_score-away_score)
            
    if team == away_team:
        if abs(away_score-home_score) >= 3:
            if away_score > home_score:
                diff = '3+'
            else:
                diff = '-3-'
        else:
            diff = str(away_score-home_score)
                       
    return diff

In [14]:
%%time

# for i in tqdm(range(len(shots))):
#     team = shots.at[i,'Ev_Team']
#     away_team = shots.at[i,'Away_Team']
#     home_team = shots.at[i,'Home_Team']
#     away_score = shots.at[i,'Away_Score']
#     home_score = shots.at[i,'Home_Score']
    
#     shots.at[i,'Score_Diff'] = score_diff(team, away_team, home_team, away_score, home_score)
    
# shots

CPU times: total: 0 ns
Wall time: 0 ns


<br>

## Saving Data to CSV for Convenience

In [15]:
#shots.to_csv('shots_with_calcs.csv',index=False)

In [16]:
shot_data = pd.read_csv('shots2022.csv').fillna('')
shot_data

,Game_Id,Date,Period,Event,Description,Time_Elapsed,Seconds_Elapsed,Strength,Ev_Zone,Type,...,Home_Goalie_Id,xC,yC,Home_Coach,Away_Coach,Distance,Shot_Angle,Score_Diff,Is_Rebound,Change_of_Angle
0,20001,2022-10-07,1,SHOT,"SJS ONGOAL - #28 MEIER, Wrist, Off. Zone, 45 ft.",0:23,23,5x5,Off,WRIST SHOT,...,8477424.0,44,8,John Hynes,David Quinn,45.705580,10.080598,0,0,
1,20001,2022-10-07,1,MISS,"SJS #44 VLASIC, Wrist, Wide of Net, Off. Zone,...",0:36,36,5x5,Off,WRIST SHOT,...,8477424.0,44,27,John Hynes,David Quinn,52.478567,30.963757,0,0,
2,20001,2022-10-07,1,BLOCK,"NSH #27 MCDONAGH BLOCKED BY SJS #62 LABANC, W...",0:56,56,5x5,Def,WRIST SHOT,...,8477424.0,-55,3,John Hynes,David Quinn,34.132096,5.042451,0,0,
3,20001,2022-10-07,1,SHOT,"NSH ONGOAL - #14 EKHOLM, Slap, Off. Zone, 56 ft.",0:59,59,5x5,Off,SLAP SHOT,...,8477424.0,-33,8,John Hynes,David Quinn,56.568542,8.130102,0,1,3.087651
4,20001,2022-10-07,1,GOAL,"NSH #44 SHERWOOD(1), Wrist, Off. Zone, 15 ft.A...",1:01,61,5x5,Off,WRIST SHOT,...,8477424.0,-74,-5,John Hynes,David Quinn,15.811388,18.434949,0,1,26.565051
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
152504,21312,2023-04-13,3,SHOT,"SEA ONGOAL - #67 GEEKIE, Wrist, Off. Zone, 10 ft.",17:14,1034,5x5,Off,WRIST SHOT,...,8475831.0,81,7,Dave Hakstol,Bruce Cassidy,10.630146,41.185925,-1,1,10.535257
152505,21312,2023-04-13,3,MISS,"VGK #9 EICHEL, Wrist, Wide of Net, Def. Zone, ...",18:33,1113,5x5,Def,WRIST SHOT,...,,55,41,Dave Hakstol,Bruce Cassidy,149.723078,15.892831,1,0,
152506,21312,2023-04-13,3,BLOCK,"SEA #4 SCHULTZ BLOCKED BY VGK #3 MCNABB, Wris...",18:43,1123,5x5,Def,WRIST SHOT,...,,75,-1,Dave Hakstol,Bruce Cassidy,14.035669,4.085617,-1,0,
152507,21312,2023-04-13,3,GOAL,"VGK #20 STEPHENSON(16), Poke, Def. Zone, 137 ft.",19:22,1162,5x5,Def,POKE,...,,47,19,Dave Hakstol,Bruce Cassidy,137.320792,7.953082,1,0,


The below code gets rid of NaN shot types. Missing values were assumed to be wrist shots as this is the most common shot type by far.

In [17]:
# Getting rid of NaN shot types. 

for i in tqdm(range(len(shot_data))):
    if shot_data.at[i,'Type'] == '':
        string1 = shot_data.at[i,'Description'].split(',')[1].lstrip().upper()
        if (string1 == 'OFF. ZONE') | (string1 == 'DEF. ZONE'):
            shot_data.at[i,'Type'] = 'WRIST SHOT'
        else:
            shot_data.at[i,'Type'] = string1
            
set(shot_data.Type)

  0%|          | 0/152509 [00:00<?, ?it/s]

{'BACKHAND',
 'BAT',
 'BETWEEN LEGS',
 'CRADLE',
 'DEFLECTED',
 'POKE',
 'SLAP SHOT',
 'SNAP SHOT',
 'TIP-IN',
 'WRAP-AROUND',
 'WRIST SHOT'}

<br>

## ML Workflow

In [18]:
import matplotlib.pyplot as plt
from sklearn.compose import make_column_transformer
from sklearn.calibration import CalibratedClassifierCV
from sklearn.model_selection import cross_val_predict
from sklearn.dummy import DummyClassifier, DummyRegressor
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn.linear_model import LogisticRegression, Ridge, LinearRegression, RidgeCV
from sklearn.metrics import (
    accuracy_score,
    classification_report,
    confusion_matrix,
    f1_score,
    make_scorer,
    precision_score,
    recall_score,
    ConfusionMatrixDisplay,
    precision_recall_curve,
    average_precision_score,
    roc_curve,
    roc_auc_score,
    mean_absolute_percentage_error,
    mean_squared_error,
    log_loss,
)
from sklearn.model_selection import (
    GridSearchCV,
    RandomizedSearchCV,
    cross_val_score,
    cross_validate,
    train_test_split,
)
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, StandardScaler

In [19]:
# Creating target variable column

for i in range(len(shot_data)):
    if shot_data.at[i,'Event'] == 'GOAL':
        shot_data.at[i,'Target'] = 1
    else:
        shot_data.at[i,'Target'] = 0
        
shot_data

,Game_Id,Date,Period,Event,Description,Time_Elapsed,Seconds_Elapsed,Strength,Ev_Zone,Type,...,xC,yC,Home_Coach,Away_Coach,Distance,Shot_Angle,Score_Diff,Is_Rebound,Change_of_Angle,Target
0,20001,2022-10-07,1,SHOT,"SJS ONGOAL - #28 MEIER, Wrist, Off. Zone, 45 ft.",0:23,23,5x5,Off,WRIST SHOT,...,44,8,John Hynes,David Quinn,45.705580,10.080598,0,0,,0.0
1,20001,2022-10-07,1,MISS,"SJS #44 VLASIC, Wrist, Wide of Net, Off. Zone,...",0:36,36,5x5,Off,WRIST SHOT,...,44,27,John Hynes,David Quinn,52.478567,30.963757,0,0,,0.0
2,20001,2022-10-07,1,BLOCK,"NSH #27 MCDONAGH BLOCKED BY SJS #62 LABANC, W...",0:56,56,5x5,Def,WRIST SHOT,...,-55,3,John Hynes,David Quinn,34.132096,5.042451,0,0,,0.0
3,20001,2022-10-07,1,SHOT,"NSH ONGOAL - #14 EKHOLM, Slap, Off. Zone, 56 ft.",0:59,59,5x5,Off,SLAP SHOT,...,-33,8,John Hynes,David Quinn,56.568542,8.130102,0,1,3.087651,0.0
4,20001,2022-10-07,1,GOAL,"NSH #44 SHERWOOD(1), Wrist, Off. Zone, 15 ft.A...",1:01,61,5x5,Off,WRIST SHOT,...,-74,-5,John Hynes,David Quinn,15.811388,18.434949,0,1,26.565051,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
152504,21312,2023-04-13,3,SHOT,"SEA ONGOAL - #67 GEEKIE, Wrist, Off. Zone, 10 ft.",17:14,1034,5x5,Off,WRIST SHOT,...,81,7,Dave Hakstol,Bruce Cassidy,10.630146,41.185925,-1,1,10.535257,0.0
152505,21312,2023-04-13,3,MISS,"VGK #9 EICHEL, Wrist, Wide of Net, Def. Zone, ...",18:33,1113,5x5,Def,WRIST SHOT,...,55,41,Dave Hakstol,Bruce Cassidy,149.723078,15.892831,1,0,,0.0
152506,21312,2023-04-13,3,BLOCK,"SEA #4 SCHULTZ BLOCKED BY VGK #3 MCNABB, Wris...",18:43,1123,5x5,Def,WRIST SHOT,...,75,-1,Dave Hakstol,Bruce Cassidy,14.035669,4.085617,-1,0,,0.0
152507,21312,2023-04-13,3,GOAL,"VGK #20 STEPHENSON(16), Poke, Def. Zone, 137 ft.",19:22,1162,5x5,Def,POKE,...,47,19,Dave Hakstol,Bruce Cassidy,137.320792,7.953082,1,0,,1.0


In [20]:
train_df, test_df = train_test_split(shot_data, test_size=0.25, random_state=17)

In [21]:
# Selecting columns to fit the model on and columns to use for model evaluation

model_columns = ['Game_Id','Date','Period','Description','Seconds_Elapsed','Strength','Ev_Zone','Type','Score_Diff','xC','yC','Distance','Shot_Angle']
eval_columns = ['Ev_Team','Event','p1_name', 'p1_ID', 'p2_name','p2_ID', 'p3_name', 'p3_ID', 'Away_Team','Home_Team','awayPlayer1', 
                'awayPlayer1_id','awayPlayer2', 'awayPlayer2_id', 'awayPlayer3', 'awayPlayer3_id','awayPlayer4', 
                'awayPlayer4_id', 'awayPlayer5', 'awayPlayer5_id','awayPlayer6', 'awayPlayer6_id', 'homePlayer1', 
                'homePlayer1_id','homePlayer2', 'homePlayer2_id', 'homePlayer3', 'homePlayer3_id','homePlayer4', 
                'homePlayer4_id', 'homePlayer5', 'homePlayer5_id','homePlayer6', 'homePlayer6_id',
                'Away_Goalie', 'Away_Goalie_Id', 'Home_Goalie', 'Home_Goalie_Id']

In [22]:
# Will have to investigate whether or not to reset the index for the y arrays

X_train = train_df[model_columns]
X_train_eval = train_df[eval_columns]
y_train = train_df['Target'].reset_index(drop=True)


X_test = test_df[model_columns]
X_test_eval = test_df[eval_columns]
y_test = test_df['Target'].reset_index(drop=True)

X_train

,Game_Id,Date,Period,Description,Seconds_Elapsed,Strength,Ev_Zone,Type,Score_Diff,xC,yC,Distance,Shot_Angle
55761,20482,2022-12-16,1,"CGY ONGOAL - #16 ZADOROV, Slap, Off. Zone, 61 ft.",713,5x5,Off,SLAP SHOT,-1,-34,-27,61.269895,26.146841
116367,20998,2023-03-04,2,"NSH #22 BARRIE BLOCKED BY CHI #15 ANDERSON, W...",966,4x5,Def,WRIST SHOT,1,-61,5,28.442925,10.124672
53206,20459,2022-12-13,1,"SEA #8 FLEURY BLOCKED BY TBL #17 KILLORN, Wri...",976,5x5,Def,WRIST SHOT,-2,-52,23,43.566042,31.865978
89180,20768,2023-01-25,3,"DAL ONGOAL - #20 SUTER, Wrist, Off. Zone, 49 ft.",632,5x5,Off,WRIST SHOT,0,-43,-19,49.769469,22.442753
97817,20840,2023-02-11,3,"CBJ ONGOAL - #27 BOQVIST, Wrist, Off. Zone, 41...",450,4x5,Off,WRIST SHOT,1,49,-9,41.000000,12.680383
...,...,...,...,...,...,...,...,...,...,...,...,...,...
25631,20221,2022-11-11,1,"PIT #16 ZUCKER, Wrist, Wide of Net, Off. Zone,...",586,5x5,Off,WRIST SHOT,0,79,16,18.867962,57.994617
125680,21079,2023-03-15,2,"NYI ONGOAL - #29 NELSON, Tip-In, Neu. Zone, 10...",28,5x5,Neu,TIP-IN,-1,-12,29,105.080921,16.020292
42297,20365,2022-12-01,2,"COL ONGOAL - #29 MACKINNON, Wrist, Off. Zone, ...",219,4x5,Off,WRIST SHOT,-1,79,1,10.049876,5.710593
34959,20301,2022-11-22,3,"BUF #72 THOMPSON(13), Wrist, Off. Zone, 20 ft....",399,5x5,Off,WRIST SHOT,3+,75,-15,20.518285,46.974934


Now to choose the features for column transformations. The unnecessary columns will be dropped.

It will be worth investigating later whether `Seconds_Elapsed` and `Period` have any effect on xG values. 

In [23]:
numeric_feats = ['Seconds_Elapsed','Distance','Shot_Angle']  # apply scaling
categorical_feats = ['Period','Strength','Type','Score_Diff']  # apply one-hot encoding
drop_feats = ['Game_Id','Date','Ev_Zone','Description','xC','yC']  # do not include these features in modeling. 

In [24]:
ct = make_column_transformer(
    (StandardScaler(), numeric_feats),  # scaling on numeric features
    (OneHotEncoder(sparse_output=False), categorical_feats), # one-hot encoding on categorical features
    ("drop", drop_feats),  # drop the drop features
)

# Train Data

transformed = ct.fit_transform(X_train)

column_names = (numeric_feats+ct.named_transformers_['onehotencoder'].get_feature_names_out().tolist())

transformed_train_df = pd.DataFrame(transformed, columns=column_names)
transformed_train_df

,Seconds_Elapsed,Distance,Shot_Angle,Period_1,Period_2,Period_3,Period_4,Strength_3x3,Strength_3x4,Strength_3x5,...,Type_TIP-IN,Type_WRAP-AROUND,Type_WRIST SHOT,Score_Diff_-1,Score_Diff_-2,Score_Diff_-3-,Score_Diff_0,Score_Diff_1,Score_Diff_2,Score_Diff_3+
0,0.324808,1.241468,-0.083752,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.051682,-0.178112,-0.824571,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,1.080412,0.475877,0.180684,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,0.092094,0.744140,-0.255019,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,-0.430796,0.364910,-0.706403,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
114376,-0.040065,-0.592175,1.388798,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
114377,-1.643210,3.136045,-0.551975,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
114378,-1.094463,-0.973507,-1.028666,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
114379,-0.577320,-0.520808,0.879279,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [25]:
# Test Data

transformed = ct.fit_transform(X_test)

column_names = (numeric_feats+ct.named_transformers_['onehotencoder'].get_feature_names_out().tolist())

transformed_test_df = pd.DataFrame(transformed, columns=column_names)
transformed_test_df

,Seconds_Elapsed,Distance,Shot_Angle,Period_1,Period_2,Period_3,Period_4,Strength_3x3,Strength_3x4,Strength_3x5,...,Type_TIP-IN,Type_WRAP-AROUND,Type_WRIST SHOT,Score_Diff_-1,Score_Diff_-2,Score_Diff_-3-,Score_Diff_0,Score_Diff_1,Score_Diff_2,Score_Diff_3+
0,-0.017593,0.123974,-0.294521,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,0.980770,0.233246,-1.215393,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.243473,0.212786,0.638550,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.530359,-0.679193,-0.485540,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,-0.872513,0.535155,-1.167614,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38123,1.399624,-0.665240,1.913587,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
38124,1.405361,-0.325514,-1.284821,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
38125,0.645113,-0.826101,-0.686323,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
38126,-1.231121,1.945798,0.059939,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


<br>

### Metrics

The chosen metrics will be accuracy, f1, and log loss. The most important metric will be log loss.

- Accuracy: # correct / # total
- f1: 2*(recallxprecision)/(recall+precision)
- log loss: -(yln(p)+(1-y)ln(1-p))

The below function will be used in `RandomizedSearchCV` to score. 

In [26]:
def my_custom_loss_func(y_true, y_pred):
    return log_loss(y_true, y_pred)
custom_logloss = make_scorer(my_custom_loss_func, greater_is_better=False)

<br>

### Attempt 0: Dummy Classifier

First, a `DummyClassifier` will be implemented as a baseline for future comparison. In this case, the classifier will always provide an xG value equal to the (total number of goals in the season)/(total number of shots in the season).

In [27]:
goals = sum(shot_data['Target'])
shots = len(shot_data['Target'])

dummy = DummyClassifier(strategy='stratified')
dummy.fit(transformed_train_df, y_train)

predictions = dummy.predict(transformed_train_df)

accuracy = accuracy_score(y_train, dummy.predict(transformed_train_df))
f1 = f1_score(y_train, dummy.predict(transformed_train_df))

y_pred_dummy = []
for i in range(len(y_train)):
    y_pred_dummy.append(goals/shots)
    
logloss = log_loss(y_train, y_pred_dummy)

print('The accuracy of the Dummy Classifier is',accuracy)
print('The f1 of the Dummy Classifier is',f1)
print('The log loss of the Dummy Classifier is',logloss)

The accuracy of the Dummy Classifier is 0.897421774595431
The f1 of the Dummy Classifier is 0.05318382058470164
The log loss of the Dummy Classifier is 0.2101393853741702


<br>

### Attempt 1: Logistic Regression

In [28]:
lr = LogisticRegression(max_iter=1000)
lr.fit(transformed_train_df, y_train)

accuracy = accuracy_score(y_train, lr.predict(transformed_train_df))
f1 = f1_score(y_train, lr.predict(transformed_train_df))
logloss = log_loss(y_train, lr.predict_proba(transformed_train_df))

print('The accuracy of the Logistic Regression is',accuracy)
print('The f1 of the Logistic Regression is',f1)
print('The log loss of the Logistic Regression is',logloss)

The accuracy of the Logistic Regression is 0.9459962756052142
The f1 of the Logistic Regression is 0.0
The log loss of the Logistic Regression is 0.20153901951434722


In [29]:
%%time

# Hyperparameter Optimization

# param_grid = {
#     "max_iter": [100, 1000, 10000],
#     "C": [0.01, 0.1, 1, 10, 100, 1000]
# }
# random_search = RandomizedSearchCV(
#     lr, param_distributions=param_grid, n_jobs=-1, n_iter=10, scoring=custom_logloss,cv=5, random_state=17
# )
# random_search.fit(transformed_train_df, y_train)

# random_search.best_params_

CPU times: total: 0 ns
Wall time: 0 ns


**The above `RandomSearchCV` along with all the rest of the hyperparameter optization searches are all commented out since running the search every single time I worked on this notebook was time consuming. The results of each search are copied into the valiation cells for each model.** 

In [30]:
#lr = LogisticRegression(C=random_search.best_params_.get('C'),max_iter=random_search.best_params_.get('max_iter'))
# Just so I don't have to run RandomSearchCV every time

lr = LogisticRegression(C=100,max_iter=1000)
lr.fit(transformed_test_df, y_test)

accuracy = accuracy_score(y_test, lr.predict(transformed_test_df))
f1 = f1_score(y_test, lr.predict(transformed_test_df))
logloss = log_loss(y_test, lr.predict_proba(transformed_test_df))

print('The accuracy of the validation Logistic Regression is',accuracy)
print('The f1 of the validation Logistic Regression is',f1)
print('The log loss of the validation Logistic Regression is',logloss)

The accuracy of the validation Logistic Regression is 0.946286193873269
The f1 of the validation Logistic Regression is 0.0
The log loss of the validation Logistic Regression is 0.2002484648533034


In [31]:
# A list of coefficients for the Logistic Regression. Some example here make complete sense. Others don't. 

lr_coefs = pd.DataFrame(columns=['Feature','Coef'])
lr_coefs['Feature'] = column_names
lr_coefs['Coef'] = list(list(lr.coef_)[0])
lr_coefs.sort_values(by='Coef',ascending=False)

,Feature,Coef
21,Type_POKE,0.499688
13,Strength_5x3,0.455438
17,Type_BAT,0.409117
23,Type_SNAP SHOT,0.333313
9,Strength_3x5,0.232245
20,Type_DEFLECTED,0.181923
24,Type_TIP-IN,0.171601
16,Type_BACKHAND,0.092732
18,Type_BETWEEN LEGS,0.069414
0,Seconds_Elapsed,0.044682


<br>

### Attempt 2: LinearSVC

Using a `SVC` model with as many features and examples as in the training set would take an absurd amount of computation time so I will use a `LinearSVC` as a replacement. I'm not sure how to get rid of the `max_iter` warnings or if they mean anything.

In [32]:
linear_svc = LinearSVC(max_iter=1000)
clf = CalibratedClassifierCV(linear_svc) 
clf.fit(transformed_train_df, y_train)

accuracy = accuracy_score(y_train, clf.predict(transformed_train_df))
f1 = f1_score(y_train, clf.predict(transformed_train_df))
logloss = log_loss(y_train, clf.predict_proba(transformed_train_df))

print('The accuracy of the LinearSVC is',accuracy)
print('The f1 of the LinearSVC is',f1)
print('The log loss of the LinearSVC is',logloss)

Liblinear failed to converge, increase the number of iterations.
Liblinear failed to converge, increase the number of iterations.
Liblinear failed to converge, increase the number of iterations.
Liblinear failed to converge, increase the number of iterations.
Liblinear failed to converge, increase the number of iterations.


The accuracy of the LinearSVC is 0.9459700474729195
The f1 of the LinearSVC is 0.0
The log loss of the LinearSVC is 0.2019722439953438


In [33]:
%%time

# Hyperparameter Optimization

# param_grid = {
#     "max_iter": [100, 1000, 10000],
#     "C": [0.01, 0.1, 1, 10, 100, 1000]
# }
# random_search = RandomizedSearchCV(
#     lr, param_distributions=param_grid, n_jobs=-1, n_iter=10, scoring=custom_logloss, cv=5, random_state=17
# )
# random_search.fit(transformed_train_df, y_train)

# random_search.best_params_

CPU times: total: 0 ns
Wall time: 0 ns


In [34]:
#linear_svc = LinearSVC(C=random_search.best_params_.get('C'),max_iter=random_search.best_params_.get('max_iter'))
linear_svc = LinearSVC(C=100,max_iter=1000)
clf = CalibratedClassifierCV(linear_svc) 
clf.fit(transformed_test_df, y_test)

accuracy = accuracy_score(y_test, clf.predict(transformed_test_df))
f1 = f1_score(y_test, clf.predict(transformed_test_df))
logloss = log_loss(y_test, clf.predict_proba(transformed_test_df))

print('The accuracy of the validation LinearSVC is',accuracy)
print('The f1 of the validation LinearSVC is',f1)
print('The log loss of the validation LinearSVC is',logloss)

Liblinear failed to converge, increase the number of iterations.
Liblinear failed to converge, increase the number of iterations.
Liblinear failed to converge, increase the number of iterations.
Liblinear failed to converge, increase the number of iterations.


The accuracy of the validation LinearSVC is 0.946286193873269
The f1 of the validation LinearSVC is 0.0
The log loss of the validation LinearSVC is 0.2070731095169319


Liblinear failed to converge, increase the number of iterations.


<br>

### Attempt 3: Decision Tree Classifier

The initial Decision Tree is sure to overfit to the data so hyperparameter optimization is especially key for this model

In [35]:
tree = DecisionTreeClassifier()
tree.fit(transformed_train_df, y_train)

accuracy = accuracy_score(y_train, tree.predict(transformed_train_df))
f1 = f1_score(y_train, tree.predict(transformed_train_df))
logloss = log_loss(y_train, tree.predict_proba(transformed_train_df))

print('The accuracy of the Decision Tree is',accuracy)
print('The f1 of the Decision Tree is',f1)
print('The log loss of the Decision Tree is',logloss)

The accuracy of the Decision Tree is 0.9999213156031159
The f1 of the Decision Tree is 0.9992709599027947
The log loss of the Decision Tree is 0.0001090797357087664


In [36]:
%%time

# Hyperparameter Optimization

# param_grid = {
#     "max_depth": [3,5,7,10,15],
#     "max_features": [None, 'sqrt'],
#     "min_samples_split": [2,5,7,10]
# }
# random_search = RandomizedSearchCV(
#     tree, param_distributions=param_grid, n_jobs=-1, scoring=custom_logloss, n_iter=17, cv=5, random_state=17
# )
# random_search.fit(transformed_train_df, y_train)

# random_search.best_params_

CPU times: total: 0 ns
Wall time: 0 ns


In [37]:
# tree = DecisionTreeClassifier(min_samples_split=random_search.best_params_.get('min_samples_split'),
#                               max_features=random_search.best_params_.get('max_features'),
#                               max_depth=random_search.best_params_.get('max_depth'))
tree = DecisionTreeClassifier(min_samples_split=2,max_features=None,max_depth=3)
tree.fit(transformed_test_df, y_test)

accuracy = accuracy_score(y_test, tree.predict(transformed_test_df))
f1 = f1_score(y_test, tree.predict(transformed_test_df))
logloss = log_loss(y_test, tree.predict_proba(transformed_test_df))

print('The accuracy of the validation Decision Tree is',accuracy)
print('The f1 of the validation Decision Tree is',f1)
print('The log loss of the validation Decision Tree is',logloss)

The accuracy of the validation Decision Tree is 0.946286193873269
The f1 of the validation Decision Tree is 0.0
The log loss of the validation Decision Tree is 0.19919003381851008


<br>

### Attempt 4: Random Forest Classifier

In [38]:
forest = RandomForestClassifier(criterion='log_loss')
forest.fit(transformed_train_df, y_train)

accuracy = accuracy_score(y_train, forest.predict(transformed_train_df))
f1 = f1_score(y_train, forest.predict(transformed_train_df))
logloss = log_loss(y_train, forest.predict_proba(transformed_train_df))

print('The accuracy of the Random Forest is',accuracy)
print('The f1 of the Random Forest is',f1)
print('The log loss of the Random Forest is',logloss)

The accuracy of the Random Forest is 0.9998688593385265
The f1 of the Random Forest is 0.9987843423292002
The log loss of the Random Forest is 0.040877457935856044


In [39]:
%%time

# Hyperparameter Optimization

# param_grid = {
#     "n_estimators": [5,10,50,100],
#     "max_depth": [3,5,7,10,15],
#     "max_features": [None, 'sqrt'],
#     "min_samples_split": [2,5,7,10]
# }
# random_search = RandomizedSearchCV(
#     forest, param_distributions=param_grid, n_jobs=-1, scoring=custom_logloss, n_iter=17, cv=5, random_state=17
# )
# random_search.fit(transformed_train_df, y_train)

# random_search.best_params_

CPU times: total: 0 ns
Wall time: 0 ns


In [40]:
# forest = RandomForestClassifier(n_estimators=random_search.best_params_.get('n_estimators'),
#                                 min_samples_split=random_search.best_params_.get('min_samples_split'),
#                                 max_features=random_search.best_params_.get('max_features'),
#                                 max_depth=random_search.best_params_.get('max_depth'),
#                                 criterion='log_loss')
forest = RandomForestClassifier(n_estimators=5,min_samples_split=2,max_features='sqrt',max_depth=3,criterion='log_loss')
forest.fit(transformed_test_df, y_test)

accuracy = accuracy_score(y_test, forest.predict(transformed_test_df))
f1 = f1_score(y_test, forest.predict(transformed_test_df))
logloss = log_loss(y_test, forest.predict_proba(transformed_test_df))

print('The accuracy of the validation Random Forest is',accuracy)
print('The f1 of the validation Random Forest is',f1)
print('The log loss of the validation Random Forest is',logloss)

The accuracy of the validation Random Forest is 0.946286193873269
The f1 of the validation Random Forest is 0.0
The log loss of the validation Random Forest is 0.203608136433831


<br>

### Attempt 5: Gradient Boosting Classifier

In [41]:
gbc = GradientBoostingClassifier(loss='log_loss')
gbc.fit(transformed_train_df, y_train)

accuracy = accuracy_score(y_train, gbc.predict(transformed_train_df))
f1 = f1_score(y_train, gbc.predict(transformed_train_df))
logloss = log_loss(y_train, gbc.predict_proba(transformed_train_df))

print('The accuracy of the Gradient Boosting Classifier is',accuracy)
print('The f1 of the Gradient Boosting Classifier is',f1)
print('The log loss of the Gradient Boosting Classifier is',logloss)

The accuracy of the Gradient Boosting Classifier is 0.946179872531277
The f1 of the Gradient Boosting Classifier is 0.0070967741935483875
The log loss of the Gradient Boosting Classifier is 0.1918158608372455


In [42]:
%%time

# Hyperparameter Optimization

# param_grid = {
#     "n_estimators": [50,100,200,250],
#     "criterion": ['friedman_mse','squared_error'],
#     "max_depth": [3,5,7,10],
#     "max_features": [None, 'sqrt'],
#     "min_samples_split": [2,5,7,10]
# }
# random_search = RandomizedSearchCV(
#     gbc, param_distributions=param_grid, n_jobs=-1, scoring=custom_logloss, n_iter=17, cv=5, random_state=17
# )
# random_search.fit(transformed_train_df, y_train)

# random_search.best_params_

CPU times: total: 0 ns
Wall time: 0 ns


In [43]:
gbc = GradientBoostingClassifier(loss='log_loss',n_estimators=250,min_samples_split=10,max_features='sqrt',
                                 max_depth=3,criterion='squared_error')
gbc.fit(transformed_test_df, y_test)

accuracy = accuracy_score(y_test, gbc.predict(transformed_test_df))
f1 = f1_score(y_test, gbc.predict(transformed_test_df))
logloss = log_loss(y_test, gbc.predict_proba(transformed_test_df))

print('The accuracy of the validation Gradient Boosting Classifier is',accuracy)
print('The f1 of the validation Gradient Boosting Classifier is',f1)
print('The log loss of the validation Gradient Boosting Classifier is',logloss)

The accuracy of the validation Gradient Boosting Classifier is 0.9467845153168275
The f1 of the validation Gradient Boosting Classifier is 0.01933301111648139
The log loss of the validation Gradient Boosting Classifier is 0.18770672552770004


<br>

## Investigation into Model Performance

Just by comparing the accuracies and log losses of the various models, the `Gradient Boosting Classifier` performed the best. f1-score in this case did not seem to be a good indicator of success - or maybe the models just all do not perform well in the f1-score. 

A couple things I want to check: 
- As a league-wide metric, how does the number of xG from each model correspond to the number of actual goals?
- As a team metric, how does the number of xG from each team correspond to the number of actual goals?
- As a skater metric, how does the number of xG from each skater correspond to the number of actual goals?
- As a goalie metric, how does the number of GSAx for each goalie correspond to other public models?

Make sure to use the test data for validation!

### League Level

The number of xG by each model should equal the number of goals in a season. I would expect each of the models to perform quite well on this regard as the model was built using data from the same season. It's cheating a bit but with so many samples hopefully it evens out the data. 

In [44]:
X_test_eval = X_test_eval.reset_index(drop=True)
frame = [transformed_test_df,X_test_eval]
eval_df = pd.concat(frame,axis=1)

In [45]:
# Dummy Classifier
y_pred_test_dummy = []
for i in range(len(y_test)):
    y_pred_test_dummy.append(goals/shots)
xG_dummy = sum(y_pred_test_dummy)

# Logistic Regression
xG_lr = sum(pd.DataFrame(lr.predict_proba(transformed_test_df))[1])

# LinearSVC
xG_svc = sum(pd.DataFrame(clf.predict_proba(transformed_test_df))[1])

# Decision Tree 
xG_tree = sum(pd.DataFrame(tree.predict_proba(transformed_test_df))[1])

# Random Forest
xG_forest = sum(pd.DataFrame(forest.predict_proba(transformed_test_df))[1])

# Gradient Boosting
xG_gbc = sum(pd.DataFrame(gbc.predict_proba(transformed_test_df))[1])

print('The Dummy Classifier overestimated the number of goals by',xG_dummy-sum(y_test))
print('The Logistic Regression overestimated the number of goals by',xG_lr-sum(y_test))
print('The Linear SVC overestimated the number of goals by',xG_svc-sum(y_test))
print('The Decision Tree overestimated the number of goals by',xG_tree-sum(y_test))
print('The Random Forest overestimated the number of goals by',xG_forest-sum(y_test))
print('The Gradient Boosting Classifier overestimated the number of goals by',xG_gbc-sum(y_test))

The Dummy Classifier overestimated the number of goals by 8.290448433832807
The Logistic Regression overestimated the number of goals by 0.10498416945711142
The Linear SVC overestimated the number of goals by 1.6608353766278015
The Decision Tree overestimated the number of goals by -9.094947017729282e-10
The Random Forest overestimated the number of goals by 22.85934264084517
The Gradient Boosting Classifier overestimated the number of goals by 0.3631325846913569


The `Decision Tree Classifier` predicted future xG the best in this case. There is virtually no difference between the predicted results and actual results. Again, in choosing which model performs best, I don't think that this section should carry much weight since the model was built on the same season data. 

<br>

### Team Level

The number of xG for each team should be similar to the number of actual goals they scored. However, some teams with higher-than-average or lower-than-average finishing will finish the season above or below their calculated xG respectively. Compare this to other public models to confirm accuracy.

In [46]:
league = list(set(eval_df.Ev_Team))
temp = eval_df.reset_index(drop=True)
listing = pd.DataFrame(league,columns=['Team'])
GF_list = []
for team in league:
    temp2 = temp.loc[temp.Ev_Team == team]
    to_predict_df = transformed_test_df.iloc[temp2.index]
    GF = sum(y_test.iloc[temp2.index])
    GF_list.append(GF)
array1 = pd.DataFrame(GF_list,columns=['GF'])
frame = [listing,array1]
listing = pd.concat(frame,axis=1)

In [47]:
league = listing['Team']
models = [dummy,lr,clf,tree,forest,gbc]
model_name = ['Dummy','LR','LinearSVC','Tree','Forest','GBC']

temp = eval_df.reset_index(drop=True)
for i in range(len(models)):
    model = models[i]
    name = model_name[i]
    xG_list = []
    GAx_list = []
    for team in league:
        temp2 = temp.loc[temp.Ev_Team == team]
        to_predict_df = transformed_test_df.iloc[temp2.index]
        xG = sum(pd.DataFrame(model.predict_proba(to_predict_df))[1])
        xG_list.append(xG)
    array1 = pd.DataFrame(xG_list,columns=[name+'_xG'])
    frame = [listing,array1]
    listing = pd.concat(frame,axis=1)

In [48]:
listing = listing.sort_values(by='Team',ascending=True)
listing

,Team,GF,Dummy_xG,LR_xG,LinearSVC_xG,Tree_xG,Forest_xG,GBC_xG
12,ANA,53.0,57.0,50.640444,55.952101,53.746064,56.450356,51.607059
1,ARI,56.0,52.0,53.689373,55.956083,56.382449,56.203264,54.138536
10,BOS,83.0,63.0,67.572698,63.554044,62.974325,65.048167,66.525532
0,BUF,74.0,78.0,62.873495,63.126562,63.624962,64.075025,64.111986
4,CAR,64.0,84.0,74.649802,75.339847,71.749596,74.133851,71.892426
11,CBJ,54.0,51.0,59.451040,61.293302,61.652804,62.043111,60.135111
8,CGY,47.0,73.0,73.104959,76.158343,71.551258,75.726296,71.425523
30,CHI,54.0,71.0,49.166508,53.950852,52.103219,53.650103,50.815137
15,COL,75.0,74.0,68.427272,69.061538,65.375488,69.640143,68.570044
3,DAL,74.0,46.0,66.720693,64.602461,63.828164,65.570178,67.311234


The best metrics for measuring model performance on total numbers such as these are Mean Average Percentage Error (MAPE) and Root Mean Squared Error (RMSE). 

In [49]:
errors = pd.DataFrame(['Dummy','LR','LinearSVC','Tree','Forest','GBC'],columns=['Model'])
models = listing.columns.drop(['Team','GF'])
mape_list = []
rmse_list = []
for i in models:
    MAPE = mean_absolute_percentage_error(listing['GF'],listing[i])
    RMSE = mean_squared_error(listing['GF'],listing[i])
    mape_list.append(MAPE)
    rmse_list.append(RMSE)
array1 = pd.DataFrame(mape_list,columns=['MAPE'])
array2 = pd.DataFrame(rmse_list,columns=['RMSE'])
frame = [errors,array1,array2]
errors = pd.concat(frame,axis=1)
errors

,Model,MAPE,RMSE
0,Dummy,0.171045,200.656250
1,LR,0.134523,97.295251
2,LinearSVC,0.136274,113.278644
3,Tree,0.129889,98.159621
4,Forest,0.133573,105.027638
5,GBC,0.123971,89.272982


On the team level, the best performing models are `Logistic Regression`, `Decision Tree Classifier`, and `Gradient Boosting Classifier`. These three models performed similarly but the best performing model was the `Gradient Boosting Classifier`. 

<br>

### Player Level

The number of xG for each player should correspond to the number of goals scored by the player. Check how this compares to other public models. Some work needs to be done to prepare for this. Also, a listing of which players accrued the most xG would be worth investigating although the numbers will be a bit off due to the train/test split. 

In [50]:
player_list = list(set(list(set(eval_df.p1_name)) + list(set(eval_df.p2_name))))

player_eval = pd.DataFrame(columns=['Player','xG','GF'])
player_eval['Player'] = player_list
player_eval['xG'] = 0
player_eval['GF'] = 0
player_eval = player_eval.set_index('Player')

# Using LR

for i in tqdm(range(len(eval_df))):
    p1 = eval_df.at[i,'p1_name']
    if eval_df.at[i,'Event'] == 'BLOCK':
        p1 = eval_df.at[i,'p2_name']
    player_eval.at[p1,'xG'] = player_eval.at[p1,'xG'] + lr.predict_proba(transformed_test_df.iloc[[i]])[0,1]
    player_eval.at[p1,'GF'] = player_eval.at[p1,'GF'] + y_test[i]
    
player_eval = player_eval.sort_values(by='xG', ascending=False)

RMSE = mean_squared_error(player_eval['GF'],player_eval['xG'])

print('RMSE of Logistic Regression is:',RMSE)
player_eval.head(n=10)

  0%|          | 0/38128 [00:00<?, ?it/s]

RMSE of Logistic Regression is: 2.5102594076846216


,xG,GF
Player,,
DAVID PASTRNAK,10.879487,13
BRADY TKACHUK,10.854948,14
MIKKO RANTANEN,10.374565,15
TIMO MEIER,10.224007,11
AUSTON MATTHEWS,10.175654,9
CONNOR MCDAVID,10.037287,10
JACK HUGHES,9.910215,19
MATTHEW TKACHUK,9.709801,8
JOHN TAVARES,9.421055,8


In [51]:
player_list = list(set(list(set(eval_df.p1_name)) + list(set(eval_df.p2_name))))

player_eval = pd.DataFrame(columns=['Player','xG','GF'])
player_eval['Player'] = player_list
player_eval['xG'] = 0
player_eval['GF'] = 0
player_eval = player_eval.set_index('Player')

# Using Tree

for i in tqdm(range(len(eval_df))):
    p1 = eval_df.at[i,'p1_name']
    if eval_df.at[i,'Event'] == 'BLOCK':
        p1 = eval_df.at[i,'p2_name']
    player_eval.at[p1,'xG'] = player_eval.at[p1,'xG'] + tree.predict_proba(transformed_test_df.iloc[[i]])[0,1]
    player_eval.at[p1,'GF'] = player_eval.at[p1,'GF'] + y_test[i]
    
player_eval = player_eval.sort_values(by='xG', ascending=False)

RMSE = mean_squared_error(player_eval['GF'],player_eval['xG'])

print('RMSE of Decision Tree is:',RMSE)
player_eval.head(n=10)

  0%|          | 0/38128 [00:00<?, ?it/s]

RMSE of Decision Tree is: 2.691019208226206


,xG,GF
Player,,
BRADY TKACHUK,10.364403,14
TIMO MEIER,9.892548,11
MIKKO RANTANEN,9.883152,15
AUSTON MATTHEWS,9.815724,9
CONNOR MCDAVID,9.631933,10
MATTHEW TKACHUK,9.420163,8
JACK HUGHES,9.350474,19
DAVID PASTRNAK,9.298165,13
JOHN TAVARES,8.904715,8


In [52]:
player_list = list(set(list(set(eval_df.p1_name)) + list(set(eval_df.p2_name))))

player_eval = pd.DataFrame(columns=['Player','xG','GF'])
player_eval['Player'] = player_list
player_eval['xG'] = 0
player_eval['GF'] = 0
player_eval = player_eval.set_index('Player')

# Using GBC

for i in tqdm(range(len(eval_df))):
    p1 = eval_df.at[i,'p1_name']
    if eval_df.at[i,'Event'] == 'BLOCK':
        p1 = eval_df.at[i,'p2_name']
    player_eval.at[p1,'xG'] = player_eval.at[p1,'xG'] + gbc.predict_proba(transformed_test_df.iloc[[i]])[0,1]
    player_eval.at[p1,'GF'] = player_eval.at[p1,'GF'] + y_test[i]
    
player_eval = player_eval.sort_values(by='xG', ascending=False)

RMSE = mean_squared_error(player_eval['GF'],player_eval['xG'])

print('RMSE of GBC is:',RMSE)
player_eval.head(n=10)

  0%|          | 0/38128 [00:00<?, ?it/s]

RMSE of GBC is: 2.353861990763322


,xG,GF
Player,,
BRADY TKACHUK,10.921929,14
MIKKO RANTANEN,10.677017,15
AUSTON MATTHEWS,10.448654,9
JOHN TAVARES,10.339801,8
JACK HUGHES,10.267103,19
DAVID PASTRNAK,10.209648,13
CONNOR MCDAVID,10.072565,10
TIMO MEIER,9.756547,11
MATTHEW TKACHUK,9.361687,8


Out of `Logistic Regression`, `Decision Tree Classifier`, and `Gradient Boosting Classifier`, the `Gradient Boosting Classifier` performed the best with the lowest RMSE. It is hard to closely visually inspect the xG and GF numbers since the sample is random. It does bode well that the NHL's top forwards occupy the top of the xG leaderboards though. 

<br>

### Goalie Level

The number of xG saved by each goalie should correspond to other public models. Again, sample distribution and season fluctuation could have an effect on the numbers here.

In [53]:
goalie_list = list(set(list(set(eval_df.Away_Goalie)) + list(set(eval_df.Home_Goalie))))

goalie_eval = pd.DataFrame(columns=['Player','xG Faced','GA','GSAx'])
goalie_eval['Player'] = goalie_list
goalie_eval = goalie_eval.reset_index(drop=True)
goalie_eval['xG Faced'] = 0
goalie_eval['GA'] = 0
goalie_eval = goalie_eval.drop(index=0).set_index('Player')

In [54]:
# Using GBC

for i in tqdm(range(len(eval_df))):
    if eval_df.at[i,'Ev_Team'] == eval_df.at[i,'Away_Team']:
        p1 = eval_df.at[i,'Home_Goalie']
        if p1 == '':
            continue
        goalie_eval.at[p1,'xG Faced'] = goalie_eval.at[p1,'xG Faced'] + gbc.predict_proba(transformed_test_df.iloc[[i]])[0,1]
        goalie_eval.at[p1,'GA'] = goalie_eval.at[p1,'GA'] + y_test[i]
    if eval_df.at[i,'Ev_Team'] == eval_df.at[i,'Home_Team']:
        p1 = eval_df.at[i,'Away_Goalie']
        if p1 == '':
            continue
        goalie_eval.at[p1,'xG Faced'] = goalie_eval.at[p1,'xG Faced'] + gbc.predict_proba(transformed_test_df.iloc[[i]])[0,1]
        goalie_eval.at[p1,'GA'] = goalie_eval.at[p1,'GA'] + y_test[i]
    
goalie_eval['GSAx'] = goalie_eval['xG Faced'] - goalie_eval['GA']
goalie_eval = goalie_eval.sort_values(by='GSAx', ascending=False)
goalie_eval.head(n=10)

  0%|          | 0/38128 [00:00<?, ?it/s]

,xG Faced,GA,GSAx
Player,,,
ANDREI VASILEVSKIY,45.451575,27,18.451575
JAKE OETTINGER,44.210197,29,15.210197
ILYA SOROKIN,50.947535,36,14.947535
LINUS ULLMARK,30.723772,18,12.723772
FILIP GUSTAVSSON,28.438643,16,12.438643
CARTER HART,42.434173,30,12.434173
SAM MONTEMBEAULT,33.655296,24,9.655296
VITEK VANECEK,35.053858,26,9.053858
IGOR SHESTERKIN,43.989023,35,8.989023


Interesting. Again, the random sample is a problem. There was no good metric for measuring GSAx (since good goalies will formulaically not match up well). The following section of this model tries to address the issue about sample.

<br>

## Testing Model on Full Season Data

In this section, I will run the same above evaluations but on the full 2021-22 NHL season data. This will allow for more direct and accurate comparisons between real-world data and the model. 

While having the model estimate match the official data would be encouraging, the real target is to have the numbers match up similarly in terms of the total number of goals, xG by each team, and xG by each player. Having these match up nicely is a good sign towards the projectability of this model for future seasons. 

First, the 2021-22 season data needs to be prepared. After scraping the play-by-play data, I saved it into a CSV file. The same data prep done on the 2022-23 data is done on this dataset. Again, this is all commented out since the CSV file with the calculations was saved and loaded later. 

In [55]:
# data2021 = pd.read_csv('pbp_2021.csv')

In [56]:
# Only shots from Periods 1, 2 and 3 and OTs
# shots2021 = data2021.loc[(data2021.Period == 1) | (data2021.Period == 2) | (data2021.Period == 3) | (data2021.Period == 4)]
# Selecting shot events
# shots2021 = shots2021.loc[(shots2021.Event == 'SHOT') | (shots2021.Event == 'MISS') | (shots2021.Event == 'BLOCK') | (shots2021.Event == 'GOAL')]
# Removing penalty shots
# shots2021 = shots2021.drop(index = shots2021.loc[shots2021.Strength == '0x0'].index)
# Selecting only regular season games
# shots2021 = shots2021.loc[shots2021.Game_Id < 30000].sort_values(by=['Game_Id','Period'],ascending=[True,True]).reset_index(drop=True)

# Removing shots with NaN locations from ML dataset

# array = []

# for i in tqdm(range(len(shots2021))):
#     if math.isnan(shots2021.at[i,'xC']) == True:
#         array.append(i)
        
# shots2021 = shots2021.drop(index = array).reset_index(drop = True)
# shots2021



In [57]:
# for j in tqdm(range(len(shots2021))):
#     shots2021.at[j,'Distance'] = get_shot_distances(shots2021,shots2021.at[j,'Game_Id'],shots2021.at[j,'Period'],
#                                                     shots2021.at[j,'Ev_Team'],shots2021.at[j,'Away_Team'],
#                                                     shots2021.at[j,'Home_Team'],shots2021.at[j,'xC'],shots2021.at[j,'yC'])
#     shots2021.at[j,'Shot_Angle'] = get_shot_angles(shots2021,shots2021.at[j,'Game_Id'],shots2021.at[j,'Period'],
#                                                    shots2021.at[j,'Ev_Team'],shots2021.at[j,'Away_Team'],
#                                                    shots2021.at[j,'Home_Team'],shots2021.at[j,'xC'],shots2021.at[j,'yC'])
    
# shots2021

In [58]:
%%time

# for i in tqdm(range(len(shots2021))):
#     team = shots2021.at[i,'Ev_Team']
#     away_team = shots2021.at[i,'Away_Team']
#     home_team = shots2021.at[i,'Home_Team']
#     away_score = shots2021.at[i,'Away_Score']
#     home_score = shots2021.at[i,'Home_Score']
    
#     shots2021.at[i,'Score_Diff'] = score_diff(team, away_team, home_team, away_score, home_score)
    
# shots2021

CPU times: total: 0 ns
Wall time: 0 ns


In [59]:
# shots2021.to_csv('shots2021.csv',index=False)

In [60]:
shot_data2021 = pd.read_csv('shots2021.csv').fillna('')

# Getting rid of NaN shot types. 

for i in range(len(shot_data2021)):
    if shot_data2021.at[i,'Type'] == '':
        string1 = shot_data2021.at[i,'Description'].split(',')[1].lstrip().upper()
        if (string1 == 'OFF. ZONE') | (string1 == 'DEF. ZONE'):
            shot_data2021.at[i,'Type'] = 'WRIST SHOT'
        else:
            shot_data2021.at[i,'Type'] = string1
            
# Creating target variable column

for i in range(len(shot_data2021)):
    if shot_data2021.at[i,'Event'] == 'GOAL':
        shot_data2021.at[i,'Target'] = 1
    else:
        shot_data2021.at[i,'Target'] = 0
        
# Selecting columns to fit the model on and columns to use for model evaluation
# Redefining these here as a refresher.

model_columns = ['Game_Id','Date','Period','Event','Description','Seconds_Elapsed','Strength','Ev_Zone','Type','Score_Diff','xC','yC','Distance','Shot_Angle']
eval_columns = ['Ev_Team','Event','p1_name', 'p1_ID', 'p2_name','p2_ID', 'p3_name', 'p3_ID', 'Away_Team','Home_Team','awayPlayer1', 
                'awayPlayer1_id','awayPlayer2', 'awayPlayer2_id', 'awayPlayer3', 'awayPlayer3_id','awayPlayer4', 
                'awayPlayer4_id', 'awayPlayer5', 'awayPlayer5_id','awayPlayer6', 'awayPlayer6_id', 'homePlayer1', 
                'homePlayer1_id','homePlayer2', 'homePlayer2_id', 'homePlayer3', 'homePlayer3_id','homePlayer4', 
                'homePlayer4_id', 'homePlayer5', 'homePlayer5_id','homePlayer6', 'homePlayer6_id',
                'Away_Goalie', 'Away_Goalie_Id', 'Home_Goalie', 'Home_Goalie_Id']

# Redefining feature categories

numeric_feats = ['Seconds_Elapsed','Distance','Shot_Angle']  # apply scaling
categorical_feats = ['Period','Strength','Type','Score_Diff']  # apply one-hot encoding
drop_feats = ['Game_Id','Date','Event','Ev_Zone','Description','xC','yC']  # do not include these features in modeling. 

In [61]:
# For some reason I have to do this to make the functions work

shot_data2021 = shot_data2021.drop(shot_data2021.loc[shot_data2021.Strength == '2x5'].index)
shot_data2021 = shot_data2021.drop(shot_data2021.loc[shot_data2021.Strength == '5x1'].index)
shot_data2021 = shot_data2021.drop(shot_data2021.loc[shot_data2021.Strength == '5x6'].index)
shot_data2021 = shot_data2021.drop(shot_data2021.loc[shot_data2021.Strength == '6x4'].index)
shot_data2021 = shot_data2021.drop(shot_data2021.loc[shot_data2021.Strength == '6x5'].index).reset_index(drop=True)

# This removes a total of about 10 shots from the dataset.

In [62]:
# This is just an easy way to deal with predict_proba issues. Should not have a great effect on overall results
shot_data2021.loc[shot_data2021.Type == 'WRIST SHOT']
shot_data2021.at[4,'Type'] = 'BAT'
shot_data2021.at[148347,'Type'] = 'POKE'
shot_data2021.at[148350,'Type'] = 'CRADLE'
shot_data2021.at[148352,'Type'] = 'BETWEEN LEGS'

shot_data2021

,Game_Id,Date,Period,Event,Description,Time_Elapsed,Seconds_Elapsed,Strength,Ev_Zone,Type,...,xC,yC,Home_Coach,Away_Coach,Distance,Shot_Angle,Score_Diff,Is_Rebound,Change_of_Angle,Target
0,20001,2021-10-12,1,SHOT,"TBL ONGOAL - #91 STAMKOS, Wrist, Off. Zone, 42...",1:03,63,5x5,Off,WRIST SHOT,...,61,-32,Jon Cooper,Mike Sullivan,42.520583,48.814075,0,0,,0.0
1,20001,2021-10-12,1,BLOCK,"TBL #24 BOGOSIAN BLOCKED BY PIT #23 MCGINN, W...",1:25,85,5x5,Def,WRIST SHOT,...,60,-17,Jon Cooper,Mike Sullivan,33.615473,30.379126,0,0,,0.0
2,20001,2021-10-12,1,SHOT,"PIT ONGOAL - #23 MCGINN, Wrist, Off. Zone, 30 ft.",1:44,104,5x5,Off,WRIST SHOT,...,-65,19,Jon Cooper,Mike Sullivan,30.610456,38.367485,0,0,,0.0
3,20001,2021-10-12,1,SHOT,"TBL ONGOAL - #44 RUTTA, Wrist, Neu. Zone, 100 ft.",2:01,121,5x5,Neu,WRIST SHOT,...,-8,-27,Jon Cooper,Mike Sullivan,100.687636,15.554571,0,0,,0.0
4,20001,2021-10-12,1,SHOT,"PIT ONGOAL - #43 HEINEN, Wrist, Off. Zone, 29 ft.",2:47,167,5x5,Off,BAT,...,-60,-4,Jon Cooper,Mike Sullivan,29.274562,7.853313,0,0,,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
148348,21312,2022-04-29,3,BLOCK,"SEA #10 BENIERS BLOCKED BY SJS #28 MEIER, Wri...",18:38,1118,5x5,Def,WRIST SHOT,...,85,6,Dave Hakstol,Bob Boughner,7.211103,56.309932,2,0,,0.0
148349,21312,2022-04-29,3,BLOCK,"SEA #10 BENIERS BLOCKED BY SJS #92 BALCERS, W...",18:53,1133,5x5,Def,WRIST SHOT,...,80,2,Dave Hakstol,Bob Boughner,9.219544,12.528808,2,0,,0.0
148350,21312,2022-04-29,3,GOAL,"SEA #49 RASK(9), Wrist, Neu. Zone, 84 ft.Assis...",19:08,1148,5x5,Neu,CRADLE,...,13,-37,Dave Hakstol,Bob Boughner,84.528102,25.958769,2,0,,1.0
148351,21312,2022-04-29,3,MISS,"SJS #53 MELOCHE, Snap, Wide of Net, Off. Zone,...",19:54,1194,5x5,Off,SNAP SHOT,...,-58,41,Dave Hakstol,Bob Boughner,51.400389,52.907163,-3-,0,,0.0


In [63]:
full_2021 = shot_data2021[model_columns]
desc_2021 = shot_data2021[eval_columns]
y_eval2021 = shot_data2021['Target']

print('There were',sum(y_eval2021),'goals scored during the 2021-22 NHL season.')

There were 8121.0 goals scored during the 2021-22 NHL season.


Now, this number does not match up exactly with the goals for column on the NHL website. However, this may be because the NHL website counts the shootout winning goal as a goal for. Additionally, this dataset does not include penalty shots.

In [64]:
ct = make_column_transformer(
    (StandardScaler(), numeric_feats),  # scaling on numeric features
    (OneHotEncoder(sparse_output=False), categorical_feats), # one-hot encoding on categorical features
    ("drop", drop_feats),  # drop the drop features
)

# Train Data

transformed = ct.fit_transform(full_2021)

column_names = (numeric_feats+ct.named_transformers_['onehotencoder'].get_feature_names_out().tolist())

transformed_2021_df = pd.DataFrame(transformed, columns=column_names)
transformed_2021_df

,Seconds_Elapsed,Distance,Shot_Angle,Period_1,Period_2,Period_3,Period_4,Strength_3x3,Strength_3x4,Strength_3x5,...,Type_TIP-IN,Type_WRAP-AROUND,Type_WRIST SHOT,Score_Diff_-1,Score_Diff_-2,Score_Diff_-3-,Score_Diff_0,Score_Diff_1,Score_Diff_2,Score_Diff_3+
0,-1.543068,0.419959,1.027745,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,-1.479825,0.028537,0.134531,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,-1.425207,-0.103548,0.521584,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,-1.376337,2.976677,-0.583751,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,-1.244102,-0.162267,-0.956894,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
148348,1.489708,-1.132061,1.390935,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
148349,1.532828,-1.043780,-0.730356,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
148350,1.575948,2.266389,-0.079645,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
148351,1.708183,0.810268,1.226064,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


It seems as the `Logistic Regression` and the `Gradient Boosting Classifier` work the best so I will evaluate this season using both those models. 

In [65]:
frame = [transformed_2021_df,desc_2021]
eval_2021 = pd.concat(frame,axis=1)

eval_2021

,Seconds_Elapsed,Distance,Shot_Angle,Period_1,Period_2,Period_3,Period_4,Strength_3x3,Strength_3x4,Strength_3x5,...,homePlayer4,homePlayer4_id,homePlayer5,homePlayer5_id,homePlayer6,homePlayer6_id,Away_Goalie,Away_Goalie_Id,Home_Goalie,Home_Goalie_Id
0,-1.543068,0.419959,1.027745,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,ZACH BOGOSIAN,8474567.0,MIKHAIL SERGACHEV,8479410.0,ANDREI VASILEVSKIY,8476883.0,TRISTAN JARRY,8477465.0,ANDREI VASILEVSKIY,8476883.0
1,-1.479825,0.028537,0.134531,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,ZACH BOGOSIAN,8474567.0,MIKHAIL SERGACHEV,8479410.0,ANDREI VASILEVSKIY,8476883.0,TRISTAN JARRY,8477465.0,ANDREI VASILEVSKIY,8476883.0
2,-1.425207,-0.103548,0.521584,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,JAN RUTTA,8480172.0,VICTOR HEDMAN,8475167.0,ANDREI VASILEVSKIY,8476883.0,TRISTAN JARRY,8477465.0,ANDREI VASILEVSKIY,8476883.0
3,-1.376337,2.976677,-0.583751,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,JAN RUTTA,8480172.0,VICTOR HEDMAN,8475167.0,ANDREI VASILEVSKIY,8476883.0,TRISTAN JARRY,8477465.0,ANDREI VASILEVSKIY,8476883.0
4,-1.244102,-0.162267,-0.956894,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,RYAN MCDONAGH,8474151.0,ERIK CERNAK,8478416.0,ANDREI VASILEVSKIY,8476883.0,TRISTAN JARRY,8477465.0,ANDREI VASILEVSKIY,8476883.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
148348,1.489708,-1.132061,1.390935,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,WILL BORGEN,8478840.0,CARSON SOUCY,8477369.0,CHRIS DRIEDGER,8476904.0,,,CHRIS DRIEDGER,8476904.0
148349,1.532828,-1.043780,-0.730356,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,WILL BORGEN,8478840.0,CARSON SOUCY,8477369.0,CHRIS DRIEDGER,8476904.0,,,CHRIS DRIEDGER,8476904.0
148350,1.575948,2.266389,-0.079645,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,WILL BORGEN,8478840.0,CARSON SOUCY,8477369.0,CHRIS DRIEDGER,8476904.0,,,CHRIS DRIEDGER,8476904.0
148351,1.708183,0.810268,1.226064,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,DENNIS CHOLOWSKI,8479395.0,DERRICK POULIOT,8476884.0,CHRIS DRIEDGER,8476904.0,KAAPO KAHKONEN,8478039.0,CHRIS DRIEDGER,8476904.0


<br>

### 2021 League Evaluation

In [66]:
# Dummy Classifier
y_2021 = []
goals = sum(y_eval2021)
shots = len(y_eval2021)
for i in range(len(y_eval2021)):
    y_2021.append(goals/shots)
xG_dummy = sum(y_2021)

# Logistic Regression
xG_lr = sum(pd.DataFrame(lr.predict_proba(transformed_2021_df))[1])

# Gradient Boosting
xG_gbc = sum(pd.DataFrame(gbc.predict_proba(transformed_2021_df))[1])

print('The Dummy Classifier overestimated the number of goals by',xG_dummy-sum(y_eval2021))
print('The Logistic Regression overestimated the number of goals by',xG_lr-sum(y_eval2021))
print('The Logistic Regression has a % error of:',abs(xG_lr-sum(y_eval2021))/goals*100)
print('The Gradient Boosting Classifier overestimated the number of goals by',xG_gbc-sum(y_eval2021))
print('The Gradient Boosting Classifier has a % error of:',abs(xG_gbc-sum(y_eval2021))/goals*100)

The Dummy Classifier overestimated the number of goals by -2.4312612367793918e-08
The Logistic Regression overestimated the number of goals by -296.1917023864762
The Logistic Regression has a % error of: 3.6472318973830338
The Gradient Boosting Classifier overestimated the number of goals by -155.6785787924291
The Gradient Boosting Classifier has a % error of: 1.9169877945133493


Ok. So the model underestimated the number of goals across the entire 2021-22 NHL season by around 300 goals for the `Logistic Regression` and by around 150 goals for the `GBC`. 

Really, a 3% error for `LR` and 1.8% error for `GBC` are really not that bad actually. 

<br>

### 2021 Team Investigation

In [67]:
league = list(set(eval_df.Ev_Team))
temp = eval_2021.reset_index(drop=True)
listing = pd.DataFrame(league,columns=['Team'])
GF_list = []
for team in league:
    temp2 = temp.loc[temp.Ev_Team == team]
    to_predict_df = transformed_2021_df.iloc[temp2.index]
    GF = sum(y_eval2021.iloc[temp2.index])
    GF_list.append(GF)
array1 = pd.DataFrame(GF_list,columns=['GF'])
frame = [listing,array1]
listing = pd.concat(frame,axis=1)

In [68]:
league = listing['Team']
models = [dummy,lr,clf,tree,forest,gbc]
model_name = ['Dummy','LR','LinearSVC','Tree','Forest','GBC']

temp = eval_2021.reset_index(drop=True)
for i in range(len(models)):
    model = models[i]
    name = model_name[i]
    xG_list = []
    GAx_list = []
    for team in league:
        temp2 = temp.loc[temp.Ev_Team == team]
        to_predict_df = transformed_2021_df.iloc[temp2.index]
        xG = sum(pd.DataFrame(model.predict_proba(to_predict_df))[1])
        xG_list.append(xG)
    array1 = pd.DataFrame(xG_list,columns=[name+'_xG'])
    frame = [listing,array1]
    listing = pd.concat(frame,axis=1)

In [69]:
listing = listing.sort_values(by='GF',ascending=False)
listing

,Team,GF,Dummy_xG,LR_xG,LinearSVC_xG,Tree_xG,Forest_xG,GBC_xG
21,FLA,337.0,259.0,296.687546,292.118268,303.023030,289.687167,306.681935
28,TOR,311.0,287.0,282.402994,275.688624,280.939863,277.469912,282.969320
15,COL,308.0,292.0,271.408031,277.247056,266.615423,274.504364,277.707917
5,STL,308.0,222.0,233.011759,232.669421,231.047709,231.403823,233.254703
19,MIN,305.0,246.0,243.959750,255.215970,243.321047,248.770640,243.286866
8,CGY,291.0,305.0,276.182709,280.981300,275.997519,277.847147,284.128322
22,T.B,285.0,257.0,250.767393,248.144907,251.380813,245.933769,258.601559
24,EDM,284.0,281.0,273.466769,267.605551,285.478773,268.744248,285.850978
4,CAR,277.0,251.0,280.024862,273.187595,288.965848,274.055240,286.923757
23,WSH,270.0,268.0,243.410027,251.227825,242.455369,247.337008,242.217437


In [70]:
errors = pd.DataFrame(['Dummy','LR','LinearSVC','Tree','Forest','GBC'],columns=['Model'])
models = listing.columns.drop(['Team','GF'])
mape_list = []
rmse_list = []
for i in models:
    MAPE = mean_absolute_percentage_error(listing['GF'],listing[i])
    RMSE = mean_squared_error(listing['GF'],listing[i])
    mape_list.append(MAPE)
    rmse_list.append(RMSE)
array1 = pd.DataFrame(mape_list,columns=['MAPE'])
array2 = pd.DataFrame(rmse_list,columns=['RMSE'])
frame = [errors,array1,array2]
errors = pd.concat(frame,axis=1)
errors

,Model,MAPE,RMSE
0,Dummy,0.071319,824.562500
1,LR,0.069871,647.912992
2,LinearSVC,0.075278,643.198037
3,Tree,0.082919,733.879156
4,Forest,0.076690,691.195796
5,GBC,0.074984,652.996660


It seems like the larger sample size has lowered the MAPE. This decrease is logical and a quick check over the numbers show that the models predict the 2021 goals quite well. The largest disparity seems to be with STL - a difference of around 70 goals out of 300. I'm not sure why this has such a big disparity while other teams are a lot closer.

<br>

### 2021 Player Evaluation

In [71]:
player_list = list(set(list(set(eval_2021.p1_name)) + list(set(eval_2021.p2_name))))

player_eval = pd.DataFrame(columns=['Player','xG','GF'])
player_eval['Player'] = player_list
player_eval['xG'] = 0
player_eval['GF'] = 0
player_eval = player_eval.set_index('Player')

# Using LR

for i in tqdm(range(len(eval_2021))):
    p1 = eval_2021.at[i,'p1_name']
    if eval_2021.at[i,'Event'] == 'BLOCK':
        p1 = eval_2021.at[i,'p2_name']
    player_eval.at[p1,'xG'] = player_eval.at[p1,'xG'] + lr.predict_proba(transformed_2021_df.iloc[[i]])[0,1]
    player_eval.at[p1,'GF'] = player_eval.at[p1,'GF'] + y_eval2021[i]
    
player_eval = player_eval.sort_values(by='xG', ascending=False)

RMSE = mean_squared_error(player_eval['GF'],player_eval['xG'])

print('RMSE of Logistic Regression is:',RMSE)
player_eval.head(n=10)

  0%|          | 0/148353 [00:00<?, ?it/s]

RMSE of Logistic Regression is: 17.942616092734095


,xG,GF
Player,,
AUSTON MATTHEWS,35.737373,59
CONNOR MCDAVID,34.594947,43
LEON DRAISAITL,34.253200,55
ALEX OVECHKIN,33.862953,50
SEBASTIAN AHO,33.408678,39
BRADY TKACHUK,31.420599,30
JOHN TAVARES,31.229987,27
KYLE CONNOR,30.280181,47
TIMO MEIER,30.218274,35


In [72]:
player_eval = pd.DataFrame(columns=['Player','xG','GF'])
player_eval['Player'] = player_list
player_eval['xG'] = 0
player_eval['GF'] = 0
player_eval = player_eval.set_index('Player')

# Using GBC

for i in tqdm(range(len(eval_2021))):
    p1 = eval_2021.at[i,'p1_name']
    if eval_2021.at[i,'Event'] == 'BLOCK':
        p1 = eval_2021.at[i,'p2_name']
    player_eval.at[p1,'xG'] = player_eval.at[p1,'xG'] + gbc.predict_proba(transformed_2021_df.iloc[[i]])[0,1]
    player_eval.at[p1,'GF'] = player_eval.at[p1,'GF'] + y_eval2021[i]
    
player_eval = player_eval.sort_values(by='xG', ascending=False)

RMSE = mean_squared_error(player_eval['GF'],player_eval['xG'])

print('RMSE of GBC is:',RMSE)
player_eval.head(n=10)

  0%|          | 0/148353 [00:00<?, ?it/s]

RMSE of GBC is: 15.751386215329369


,xG,GF
Player,,
AUSTON MATTHEWS,37.940055,59
CONNOR MCDAVID,37.738719,43
LEON DRAISAITL,35.815372,55
ALEX OVECHKIN,35.461957,50
BRADY TKACHUK,34.328449,30
SEBASTIAN AHO,34.036007,39
MATTHEW TKACHUK,33.059786,42
CHRIS KREIDER,32.709787,52
JOHN TAVARES,31.937919,27


The overall underestimation of actual goals is very apparent in these listings. 

<br>

### 2021 Goalie Evaluation

In [73]:
goalie_list = list(set(list(set(eval_2021.Away_Goalie)) + list(set(eval_2021.Home_Goalie))))

goalie_eval = pd.DataFrame(columns=['Player','xG Faced','GA','GSAx'])
goalie_eval['Player'] = goalie_list
goalie_eval = goalie_eval.reset_index(drop=True)
goalie_eval['xG Faced'] = 0
goalie_eval['GA'] = 0
goalie_eval = goalie_eval.drop(index=1).set_index('Player')

In [74]:
# Using GBC

for i in tqdm(range(len(eval_2021))):
    if eval_2021.at[i,'Ev_Team'] == eval_2021.at[i,'Away_Team']:
        p1 = eval_2021.at[i,'Home_Goalie']
        if p1 == '':
            continue
        goalie_eval.at[p1,'xG Faced'] = goalie_eval.at[p1,'xG Faced'] + gbc.predict_proba(transformed_2021_df.iloc[[i]])[0,1]
        goalie_eval.at[p1,'GA'] = goalie_eval.at[p1,'GA'] + y_eval2021[i]
    if eval_2021.at[i,'Ev_Team'] == eval_2021.at[i,'Home_Team']:
        p1 = eval_2021.at[i,'Away_Goalie']
        if p1 == '':
            continue
        goalie_eval.at[p1,'xG Faced'] = goalie_eval.at[p1,'xG Faced'] + gbc.predict_proba(transformed_2021_df.iloc[[i]])[0,1]
        goalie_eval.at[p1,'GA'] = goalie_eval.at[p1,'GA'] + y_eval2021[i]
    
goalie_eval['GSAx'] = goalie_eval['xG Faced'] - goalie_eval['GA']
goalie_eval = goalie_eval.sort_values(by='GSAx', ascending=False)
goalie_eval.head(n=10)

  0%|          | 0/148353 [00:00<?, ?it/s]

KeyError: 'JOONAS KORPISALO'

These results match up slightly well with other public models. The biggest outlier is James Reimer. 

<br>

## Conclusions

Overall, I think the model performed reasonably. The two best performing models were `Logistic Regression` and `Gradient Boosting Classifier`. Both models show a tendency to underestimate the number of goals scored which has the effect of undervaluing skaters and overvaluing goaltenders. I doubt this is an effect of the model rather it is the data itself that is missing additional parameters that provide telling information on the quality of a scoring chance. Such data could include:

- Rebound shots
- Rush shots
- Change of Angle between shots
- Others

There are obvious difficulties in getting the above parameters as they are not included in the official NHL play-by-play data. If they are to be added, they will have to be engineered and calculated. It would also be worth investigating how significant the features that represent the period and time in each period matter. 

Another aspect that this model does not include is a normalization for shot locations tracked. Studies have shown that different stadiums around the NHL track distances slightly differently (since this tracking is done manually as far as I know). This leads to systematic undervaluation and overvaluation of shot quality in certain NHL arenas.

In order to make the xG model more accurate and more projectable for future seasons, the suggestions above as well as other avenues of improvement should be investigated. 

<br>

#####  Aaron Foong